# Test problem from Caroe et al. (1999)

$$
\begin{align*}
\max_{x}      \quad & \frac{3}{2} x_1 + 4 x_2 + Q(x_1, x_2) \\
\mathrm{s.t.} \quad & 0 \leq x \leq 5 \\
                    & x \ \mathrm{integer}
\end{align*}
$$

where $Q(x_1,x_2)$:
$$
\begin{align*}
\max_{y}      \quad & 16 y_1 + 19 y_2 + 23 y_3 + 28 y_4 \\
\mathrm{s.t.} \quad & 2 y_1 + 3 y_2 + 4 y_3 + 5 y_4 \leq \zeta_1 - x_1 \\
                    & 6 y_1 + y_2 + 3 y_3 + 2 y_4 \leq \zeta_2 - x_2 \\
                    & y_i \in \{0,1\}
\end{align*}
$$
and $\zeta = (\zeta_1,\zeta_2)$ is uniformly distributed on
$$\Pi = \{(5,5), (5,6),\dots,(5,15),(6,5),\dots,(15,15)\}$$
giving 121 scenarios.

In [1]:
RELAX_LP = False

In [2]:
from __future__ import division
from cobra import Model
from dynamicme.optimize import Variable, Constraint
from six import iteritems
from dynamicme.decomposition import LagrangeMaster, LagrangeSubmodel

import numpy as np
import pandas as pd

In [3]:
zetas = [(i,j) for i in np.arange(5,16) for j in np.arange(5,16)]
print('Scenarios: %s'%len(zetas))

Scenarios: 121


In [4]:
cx = -np.array([3/2, 4])
fy = -np.array([16, 19, 23, 28])
xl = [0, 0]
xu = [5, 5]
A  = [[2,3,4,5],
      [6,1,3,2]]

sub_dict = {}
for k,zeta in enumerate(zetas):
    mdl = Model('sub')    
    ys  = [Variable('y_%d'%j, lower_bound=0, upper_bound=1, objective_coefficient=fy[j]) for j in range(len(fy))]
    xs  = [Variable('x_%d'%j, lower_bound=xl[j], upper_bound=xu[j], objective_coefficient=cx[j]) for j in range(len(cx))]
    for x in xs:
        x.variable_kind = 'integer'
    for y in ys:
        y.variable_kind = 'integer'
    mdl.add_reactions(xs+ys)
    for i,ai in enumerate(A):
        cons = Constraint('cons_%d'%i)
        cons._constraint_sense = 'L'
        cons._bound = zeta[i]
        xs[i].add_metabolites({cons:1})
        for j,aij in enumerate(ai):
            ys[j].add_metabolites({cons:aij})
    sub_id = 'scen_%d'%k
    sub = LagrangeSubmodel(mdl, sub_id, first_stage_vars=xs)
    sub._weight = 1./len(zetas)
    sub_dict[sub_id] = sub

In [ ]:
master = LagrangeMaster(mdl, first_stage_vars=xs)
master._INF = 1e8
master.add_submodels(sub_dict)
master._z.LB = -master._INF
master._z.UB = master._INF
master.model.Params.Presolve = 0
master.model.update()
master.model.Params.NodefileStart = 3.
for sub in sub_dict.values():
    sub.model.Params.NodefileStart = 3.

In [ ]:
master.gaptol = 1e-3
master.penaltytol = 1e-6 #1e-4
master.delta_mult = 0.5
master.delta_min = 0
master.bundle_mult = 0.5

feasible_methods = ['heuristic'] #['heuristic','enumerate']
alt_method = 'pool'
max_alt = 10


if RELAX_LP:
    master.verbosity=2
    master.max_iter = 100
    sol_master = master.solve_relaxed(feasible_methods=None)
    df_phase1 = pd.DataFrame(master.log_rows)

master.gaptol = 0.01
master.verbosity=2
master.max_iter = 100
sol_master = master.optimize(feasible_methods=feasible_methods, max_alt=max_alt, alt_method=alt_method,
                             early_heuristics=['average']) 
df_phase2 = pd.DataFrame(master.log_rows)

    Iter                    UB                    LB       gap relgap(%)   penalty                       Time(s)
  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Best   Feasible        Sub       Best                                   total    master       sub
Best Heuristic solution has objval=-56.2809917355
       0     -56.28     -56.28     -70.58     -70.58      14.3      25.4         0  0.264517  0.004324  0.133975
pred_descent=0.058876570345. Updating D_prox from -70.5785123967 to -70.5196366345.
pred_descent=0.117750990859. Updating D_prox from -70.5196366345 to -70.4018851099.
pred_descent=0.235503363338. Updating D_prox from -70.4018851099 to -70.1663820664.
pred_descent=0.471005938393. Updating D_prox from -70.1663820664 to -69.7075336344.
pred_descent=0.930486894344. Updating D_prox from -69.7075336344 to -68.927310292.
pred_descent=1.42526096512. Updating D_prox from -68.927310292 to -67.905667372

In [ ]:
master._z.X

In [ ]:
master.solve_lagrangian(master.uopt)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

xcol = 'iter'
df_phase1.loc[:,'phase'] = 1
df_phase1.loc[:,'feasUB'] = np.nan
df_phase1.loc[ df_phase1.bestUB>1e6,'bestUB'] = np.nan
df_phase2.loc[:,'phase'] = 2
df_phase2.loc[ df_phase2.feasUB>1e6,'feasUB'] = np.nan
dsplot = pd.concat([df_phase1, df_phase2])
dsplot = dsplot[[xcol,'feasUB','bestUB','bestLB','phase']]
dsplot = dsplot.melt(id_vars=[xcol,'phase'])

g = sns.FacetGrid(dsplot, hue='variable', size=3, aspect=3, row='phase', sharey=False)
g.map(plt.plot, xcol,'value', marker='o', markersize=4)
g.add_legend()

In [ ]:
objfun = master.model.getObjective()

In [ ]:
for i in [10, 100, 1000, 10000]:
    cons = master.model.getConstrs()[i]
    print("%s %s %s" % (master.model.getRow(cons), cons.Sense, cons.RHS))

for i in range(len(master.model.getConstrs())):
    cons = master.model.getConstrs()[i]
    print("%s %s %s" % (master.model.getRow(cons), cons.Sense, cons.RHS))